In [ ]:
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 28.4 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=163bfe90713c6164cd27dcd79806da1b59d0393915c04573f1f75157c63cffb4
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [ ]:
!unzip /content/drive/MyDrive/sentence_transformers.zip -d /content/

In [ ]:
import pandas as pd

dataframe = pd.read_csv('out.csv', header=None, index_col=0)
dataframe.head()

,1,2
0,,
155500,курский губернатор роман старовойт сегодня со ...,24945
23628,в результате обстрела села плехово в курской о...,24945
64741,в результате обстрела села плехово суджанского...,24945
89718,тридцать летний мужчина погиб в результате обс...,24945
66100,мужчина погиб в результате обстрела всу села п...,24945


In [ ]:
from itertools import combinations

dataset_raw = {'first': [], 'second': [], 'similarity': []}

for (first_index, first_text, first_label), (second_index, second_text, second_label) in combinations(dataframe.itertuples(), r=2):
    dataset_raw['first'].append(first_text)
    dataset_raw['second'].append(second_text)
    if first_label == second_label:
        dataset_raw['similarity'].append(1)
    else:
        dataset_raw['similarity'].append(0)

In [ ]:
dataset = pd.DataFrame.from_dict(dataset_raw)
dataset.head()

,first,second,similarity
0,курский губернатор роман старовойт сегодня со ...,в результате обстрела села плехово в курской о...,1
1,курский губернатор роман старовойт сегодня со ...,в результате обстрела села плехово суджанского...,1
2,курский губернатор роман старовойт сегодня со ...,тридцать летний мужчина погиб в результате обс...,1
3,курский губернатор роман старовойт сегодня со ...,мужчина погиб в результате обстрела всу села п...,1
4,курский губернатор роман старовойт сегодня со ...,один человек погиб в результате обстрела села ...,1


In [ ]:
import torch
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('/content/content/model_output', device=device)

In [ ]:
from sklearn.utils import shuffle

dataset_reduced = shuffle(dataset)[:len(dataset) // 8]

In [ ]:
from sentence_transformers import InputExample

input_examples = []

for index, first_text, second_text, similarity in dataset_reduced.itertuples():
    input_examples.append(InputExample(texts=[first_text, second_text], label=float(similarity)))

In [ ]:
from sklearn.model_selection import train_test_split

train_dataset, val_dataset = train_test_split(dataset_reduced, test_size=0.2, shuffle=False)
val_dataset, test_dataset = train_test_split(val_dataset, test_size=0.5, shuffle=False)

In [ ]:
test_texts_1 = test_dataset['first'].values
test_texts_2 = test_dataset['second'].values

In [ ]:
embeddings_1 = model.encode(test_texts_1, convert_to_tensor=True, device=device, show_progress_bar=True)
embeddings_2 = model.encode(test_texts_2, convert_to_tensor=True, device=device, show_progress_bar=True)

Batches:   0%|          | 0/231 [00:00<?, ?it/s]

Batches:   0%|          | 0/231 [00:00<?, ?it/s]

In [ ]:
embeddings_1.shape

torch.Size([7378, 512])

In [ ]:
from sentence_transformers import util

cosine_scores = util.cos_sim(embeddings_1, embeddings_2).cpu().numpy()

In [ ]:
cosine_scores[cosine_scores < 0.5] = 0
cosine_scores[cosine_scores >= 0.5] = 1

In [ ]:
scores = [cosine_scores[i][i] for i in range(len(cosine_scores))]
labels = list(test_dataset['similarity'])

In [ ]:
correct = 0

for i in range(len(test_dataset)):
    if scores[i] == labels[i]:
        correct += 1

In [ ]:
correct / len(labels)

0.9985090810517756